<a href="https://colab.research.google.com/github/nafis-neehal/CT-LLM/blob/main/CT_Experiment_04_18_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install nltk
# !pip3 install evaluate
# !pip3 install rouge-score
# !pip3 install openai
# !pip3 install langchain
# !pip3 install requests
# !pip3 install bert-score
# !pip3 install langchain_openai
# !pip3 install huggingface-hub

In [3]:
import numpy as np
import pandas as pd
import requests
import urllib.parse
import json
import os
import requests
from tqdm import tqdm

# Import Colab Secrets userdata module
from google.colab import userdata

In [ ]:
final_df = pd.read_csv('API_1791_CKD_Diab_Obese_Cancer_Hyp.csv')

In [ ]:
test_final_df = final_df.sample(n=100, random_state=42, ignore_index=True)

In [ ]:
test_final_df.head()

,NCTId,BriefTitle,InclusionCriteria,ExclusionCriteria,BriefSummary,Conditions,StudyType,Interventions,PrimaryOutcomes,BaselineMeasures
0,NCT00385580,Phase II Study of Dasatinib (BMS-354825) for A...,"* males, 18 or older\n* proven advanced prosta...",* symptomatic CNS (brain or spinal cord) metas...,The purpose of this study is to learn if men w...,"Prostate Cancer,",INTERVENTIONAL,"dasatinib, dasatinib,","Number of Participants With a Response, Percen...","Age Continuous, Age, Customized, Sex: Female, ..."
1,NCT02317744,Treatment of Binge Eating Disorder in Obesity:...,* Binge eating disorder (full criteria as desc...,* Medical status judged by study physician as ...,This study will test the effectiveness of the ...,"Binge Eating Disorder,",INTERVENTIONAL,"Naltrexone and bupropion combination, Pill Pla...","Binge Eating Frequency (Continuous), Binge Eat...","Age, Categorical, Age, Continuous, Sex: Female..."
2,NCT02451917,Glargine Versus NPH in Patients With Chronic K...,* Patients with type 2 diabetes mellitus and c...,"* Patients with systemic neoplasias,\n* HIV, c...",Chronic kidney disease (CKD) is one of the mos...,"Type 2 Diabetes Mellitus, Chronic Kidney Disea...",INTERVENTIONAL,"Glargine insulin, NPH insulin,","Difference in A1c Levels, Number of Hypoglycem...","Age, Continuous, Sex: Female, Male, Region of ..."
3,NCT02352974,GAD-Alum (Diamyd) Administered Into Lymph Node...,1. Informed consent given by patients\n2. Type...,1. Previous or current treatment with immunosu...,The objectives of the main study is to:\n\n* E...,"Diabetes Mellitus, Type 1,",INTERVENTIONAL,"GAD-Alum, Vitamin D,",Number of Subjects With Injection Site Reactio...,"Age, Categorical, Age, Continuous, Sex: Female..."
4,NCT02347527,Neuronal and Behavioral Effects of Implicit Pr...,* Overweight/obese adults\n* Healthy weight ad...,"* Vegetarian, vegan, or having other wide-rang...",The overall goals of this project are to deter...,"Feeding Behavior, Obesity, Overweight,",INTERVENTIONAL,"Active Implicit Priming, Control Implicit Prim...","Change in Food Image Ratings, Neuronal Respons...","Age, Continuous, Sex: Female, Male, Ethnicity ..."


## Helper Functions

In [ ]:
# prompt: Randomly get 5 rows from final_df that are not present in test_final_df. Use "NCTId" feature as unique key to check which rows are in final_df but not in test_final_df. Write this as a function where number of rows to return would be an argument to the function. Make sure to generate the random rows using a seed value.

import numpy as np
def generate_K_shot_examples(df, test_df, K, seed_value):
  """
  Returns a K number of random rows from main data that are not present in test data.
  These rows are going to be used as K-shot examples for each items in test data.

  Args:
      df: The dataframe to select rows from.
      test_df: The dataframe to check for existing rows.
      k: The number of rows to return.
      seed_value: The seed value for random number generation.

  Returns:
      A dataframe containing the K number of examples for K-shot learning.
  """

  # Get all NCTIds from the test dataframe
  test_df_nctids = set(test_df['NCTId'])

  # Filter out rows from the main dataframe that are present in the test dataframe
  df_without_test_rows = df[~df['NCTId'].isin(test_df_nctids)]

  # Set the random seed
  np.random.seed(seed_value)

  # Randomly select the specified number of rows
  random_rows = df_without_test_rows.sample(n=K, random_state=seed_value)

  return random_rows



In [ ]:
def row_to_query_converter(row):
    '''
    Returns a <trial_info> string and a <query> string from the row information.
    Args:
        row: The row information to convert.

    Returns:
        A tuple containing the <trial_info> string and the <query> string.
    '''


    title = row['BriefTitle']
    brief_summary = row['BriefSummary']
    condition = row['Conditions']
    inclusion = row['InclusionCriteria']
    exclusion = row['ExclusionCriteria']
    intervention = row['Interventions']
    outcome = row['PrimaryOutcomes']

    trial_info = f"\n<Title> - {title}\n<Brief Summary> - {brief_summary}\n<Condition> - {condition}\n<Inclusion Criteria> - {inclusion}\n<Exclusion Criteria> - {exclusion}\n<Intervention> - {intervention}\n<Outcome> - {outcome}"

    query = "Return a list of probable baseline features (seperated by comma, without itemizing or bullet points) that needs to\
    be measured before the trial starts and in each follow up visits. These baseline\
    features are usually found in Table 1 of a trial related publications. Don't give any additional explanations."

    return (trial_info, query)


## Few Shot Examples

In [14]:
# from langchain_core.messages import HumanMessage, SystemMessage
# from langchain_core.prompts.chat import (
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
#     SystemMessagePromptTemplate,
# )
from langchain_openai import ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint

# from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY_BS1")


*Step 1:* Create a list of few-shot examples. Each example should be a dictionary with the keys being the input variables and the values being the values for those input variables.

In [ ]:
def few_shot_examples(K,seed):

  k_shots = generate_K_shot_examples(final_df, test_final_df, K, seed_value = seed)

  examples = []

  for index, row in k_shots.iterrows():

    trial_info, query = row_to_query_converter(row)

    baseline_features = row['BaselineMeasures']
    answer = f"{baseline_features}"

    examples.append({"trial_info": trial_info, "query": query, "answer": answer})

  return examples


*Step 2:* Configure a formatter that will format the few-shot examples into a string. This formatter should be a PromptTemplate object.

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["trial_info", "query", "answer"], template="##Trial Info: {trial_info}\n##Question: {query}\n##Answer: {answer}"
)



*Step 3:* Create the final few-shot prompt template

In [ ]:
def get_final_prompt(K, seed):

  examples = few_shot_examples(K=K, seed=seed)

  system_message = "System: You are a helpful assistant with 10 years of experience in clinical domain and clinical trial design.\
  Answer the question based on the context below and your knowledge on clinical trial design, and clinical domain.\
  You will be given zero or few examples with corresponding answers. Follow a similar pattern to answer the final query to the best of your ability."
  final_prompt = FewShotPromptTemplate(
      examples=examples,
      example_prompt=example_prompt,
      prefix= system_message,
      suffix="##Trial Info: {trial_info}\n##Question: {query}\n##Answer: ",
      input_variables=["trial_info", "query"],
  )

  return final_prompt


In [ ]:
def get_K_response(K, index, trial_info, query, chat_llm):
  final_prompt = get_final_prompt(K=K, seed=index) #K = number of K-shot examples, all randomly picked
  prompt = final_prompt.format(trial_info=trial_info, query=query)
  ret = chat_llm.invoke(prompt)
  return ret.content

Example

In [ ]:
r = next(test_final_df.iterrows())[1]
trial_info, query = row_to_query_converter(r)
final_prompt = get_final_prompt(K=2, seed=1) #K = number of K-shot examples, all randomly picked
prompt = final_prompt.format(trial_info=trial_info, query=query)
print(prompt)

System: You are a helpful assistant with 10 years of experience in clinical domain and clinical trial design.  Answer the question based on the context below and your knowledge on clinical trial design, and clinical domain.  You will be given zero or few examples with corresponding answers. Follow a similar pattern to answer the final query to the best of your ability.

##Trial Info: 
<Title> - Treatment With Radiolabeled Monoclonal Antibody HuJ591-GS (177Lu-J591) in Patients With Metastatic Prostate Cancer
<Brief Summary> - The purpose of this study is to find out how effective 177Lu -J591 is in the treatment of patients with metastatic, androgen-independent prostate cancer.
<Condition> - Prostate Cancer, 
<Inclusion Criteria> - * Histologic diagnosis of prostate adenocarcinoma.
* Metastatic prostate cancer progressive on imaging studies and/or rising PSA despite adequate medical or surgical castration therapy.
* Progressed following discontinuation of anti-androgen therapy, if receiv

In [ ]:
%%time

from tqdm import tqdm

# initialize the models
model_name = "gpt-4-0125-preview"
chat_llm = ChatOpenAI(model_name=model_name, temperature=0)

k_shot_response_df = pd.DataFrame(columns=['TrialID', 'BaselineMeasures', 'ZeroShot', 'OneShot', 'TwoShot', 'ThreeShot'])

for index, row in tqdm(test_final_df.iterrows()):

  if index <= 44:
    continue

  trial_info, query = row_to_query_converter(row)

  trial_id = row['NCTId']
  baseline_features = row['BaselineMeasures']

  zero_shot = get_K_response(K=0, index=index, trial_info=trial_info, query=query, chat_llm=chat_llm)
  one_shot = get_K_response(K=1, index=index, trial_info=trial_info, query=query, chat_llm=chat_llm)
  two_shot = get_K_response(K=2, index=index, trial_info=trial_info, query=query, chat_llm=chat_llm)
  three_shot = get_K_response(K=3, index=index, trial_info=trial_info, query=query, chat_llm=chat_llm)

  k_shot_response_df.loc[index] = [trial_id, baseline_features, zero_shot, one_shot, two_shot, three_shot]


100it [18:40, 11.21s/it]

CPU times: user 9.93 s, sys: 915 ms, total: 10.8 s
Wall time: 18min 40s


In [ ]:
# K_final_shot_response_df = pd.concat([k_save_0_43, k_save_45_100])
# K_final_shot_response_df = K_shot_response_df.drop_duplicates(subset='TrialID', keep='first')
# K_final_shot_response_df.to_csv('final_k100_shot_response_df.csv', index=False)

## Evaluation

In [4]:
K_final_shot_response_df = pd.read_csv('final_K100_shot_response_df.csv')

In [5]:
K_final_shot_response_df.head()

,TrialID,BaselineMeasures,ZeroShot,OneShot,TwoShot,ThreeShot
0,NCT00385580,"Age Continuous, Age, Customized, Sex: Female, ...","Age, race, baseline PSA level, Gleason score, ...","Age, Sex: Male, Race/Ethnicity, Prostate Cance...","Age, Continuous, Sex: Male, Race/Ethnicity, Cu...","Age, Continuous, Sex: Male, Ethnicity (NIH/OMB..."
1,NCT02317744,"Age, Categorical, Age, Continuous, Sex: Female...","Age, sex, BMI, duration of binge eating disord...","Age, Sex: Female, Male, BMI (kg/m2), Binge Eat...","Age Continuous, Sex: Female, Male, Race (NIH/O...","Age, Continuous, Sex: Female, Male, Race (NIH/..."
2,NCT02451917,"Age, Continuous, Sex: Female, Male, Region of ...","Age, sex, duration of diabetes, baseline A1c l...","Age, Sex: Female, Male, Ethnicity (NIH/OMB), R...","Age, Continuous, Sex: Female, Male, Race/Ethni...","Age, Continuous, Sex: Female, Male, Ethnicity ..."
3,NCT02352974,"Age, Categorical, Age, Continuous, Sex: Female...","Age, gender, duration of Type 1 diabetes, fast...","Age, Sex: Female, Male, Body Mass Index (BMI) ...","Age, Continuous, Sex: Female, Male, Duration o...","Age, Continuous, Sex: Female, Male, Duration o..."
4,NCT02347527,"Age, Continuous, Sex: Female, Male, Ethnicity ...","Age, sex, BMI (Body Mass Index), baseline diet...","Age, Sex: Female, Male, BMI (Body Mass Index),...","Age, Continuous, Sex: Female, Male, Race/Ethni...","Age, Continuous, Sex: Female, Male, BMI (Body ..."


In [6]:
df = pd.melt(K_final_shot_response_df, id_vars=['TrialID', 'BaselineMeasures'], value_vars=['ZeroShot', 'OneShot', 'TwoShot', 'ThreeShot'], var_name='K-Shot', value_name='GPT_4_Response')
df.head()

,TrialID,BaselineMeasures,K-Shot,GPT_4_Response
0,NCT00385580,"Age Continuous, Age, Customized, Sex: Female, ...",ZeroShot,"Age, race, baseline PSA level, Gleason score, ..."
1,NCT02317744,"Age, Categorical, Age, Continuous, Sex: Female...",ZeroShot,"Age, sex, BMI, duration of binge eating disord..."
2,NCT02451917,"Age, Continuous, Sex: Female, Male, Region of ...",ZeroShot,"Age, sex, duration of diabetes, baseline A1c l..."
3,NCT02352974,"Age, Categorical, Age, Continuous, Sex: Female...",ZeroShot,"Age, gender, duration of Type 1 diabetes, fast..."
4,NCT02347527,"Age, Continuous, Sex: Female, Male, Ethnicity ...",ZeroShot,"Age, sex, BMI (Body Mass Index), baseline diet..."


### 1. Hand Scored Examples

In [7]:
import textwrap
def print_hand_score_examples(TrialId):
  width = 150
  testdf = df.loc[df['TrialID'] == TrialId]

  #print("\nBaseline Feature:", textwrap.fill(testdf['BaselineMeasures'].values[0], width))
  print("\nZero Shot:", textwrap.fill(testdf[testdf['K-Shot'] == 'ZeroShot'].GPT_4_Response.values[0], width))
  print("\nOne Shot:", textwrap.fill(testdf[testdf['K-Shot'] == 'OneShot'].GPT_4_Response.values[0], width))
  print("\nTwo Shot:", textwrap.fill(testdf[testdf['K-Shot'] == 'TwoShot'].GPT_4_Response.values[0], width))
  print("\nThree Shot:", textwrap.fill(testdf[testdf['K-Shot'] == 'ThreeShot'].GPT_4_Response.values[0], width))

In [8]:
def print_trial(TrialId):
  row = next(test_final_df.loc[test_final_df['NCTId'] == TrialId].iterrows())[1]

  print("Trial ID:", TrialId)

  trial_info, _ = row_to_query_converter(row)
  print(textwrap.fill(trial_info, width=150))

  print_hand_score_examples(TrialId)

**Example 1**

---
Reported in Paper: Age, Race, Sex, ECOG Performance Status, Prior Therapy (Surgery/Radiotherapy/Bisphosphonates), Bone Metastatis, RECIST-evaluable
diseases, Disease Sites (Lymph/Pelvis/Adrenal Gland/Visceral,Lung/Visceral,Liver/Visceral,Diaphragm)


Zero Shot Matches: Age, Race, Performance Status, Bone Metastases, presence of visceral metastases (5)

One Shot Matches: Age, Sex, Race, (ECOG) performance status, Presence of Metastatic Disease (5)

Two Shot Matches: Age, Sex, Race, (ECOG) performance status, Metastatic Sites (5)

Three Shot Matches: Age, Sex, Race, Metastatic Sites (4)

Total features in reference = 8

Total features in responses = variable for each shot

Precision = TP / (TP + FP)

Recall = TP / (TP + FN)

Zero Shot: Precision = 5/16 = 0.3125, Recall = 5/8 = 0.625

One Shot: Prec = 5/10 = 0.5, Recall = 5/8 = 0.625

Two Shot: 5/9 = 0.56, Recall = 5/8 = 0.625

Three Shot: 4/9 = 0.44, Recall = 4/8 = 0.5


In [ ]:
print_trial('NCT00385580')
base = "Reported in Paper: Age, Race, Sex, ECOG Performance Status, Prior Therapy (Surgery/Radiotherapy/Bisphosphonates), \
Bone Metastatis, RECIST-evaluable diseases, Disease Sites (Lymph/Pelvis/Adrenal Gland/Visceral,Lung/Visceral,Liver/Visceral,Diaphragm)"
print("")
print(textwrap.fill(base, width=150))

Trial ID: NCT00385580
 <Title> - Phase II Study of Dasatinib (BMS-354825) for Androgen-deprived Progressive Prostate Cancer <Brief Summary> - The purpose of this study is
to learn if men with metastatic prostate cancer and rising Prostate Specific Antigen (PSA), who have been surgically castrated or are undergoing
androgen deprivation with Luteinizing Hormone Releasing Hormone (LHRH) treatment, respond to dasatinib. The safety of this treatment will also be
studied. <Condition> - Prostate Cancer,  <Inclusion Criteria> - * males, 18 or older * proven advanced prostate cancer * documented metastatic disease
* rising PSA levels * castrate levels of testosterone <Exclusion Criteria> - * symptomatic CNS (brain or spinal cord) metastasis * medical condition
which may increase the risk of toxicity * any prior or ongoing anti-cancer medical therapy or immunotherapy for prostate cancer other than primary
androgen deprivation agents * unable to take oral medication <Intervention> - dasatinib, da

**Example 2**

---
Zero Shot Matches: Age, sex, BMI, Depression score, number of binge eating episodes per week (5)

One Shot Matches: Age, Sex, BMI (kg/m2), Depression scale score, Number of binge eating episodes per week (5)

Two Shot Matches: Age, Sex, Race, BMI (kg/m2), Current Depression Scale Score, Number of Binge Eating Episodes per Week (6)

Three Shot Matches: Age, Sex, Race, Body Mass Index (BMI), Beck Depression Inventory (BDI) Score, Number of Binge Eating Episodes per Week (6)

Total features in reference = 9

Total features in responses = variable for each shot

Precision = TP / (TP + FP)

Recall = TP / (TP + FN)

Zero Shot: Precision = 5/19 = 0.263, Recall = 5/9 = 0.55

One Shot: Prec = 5/13 = 0.38, Recall = 5/9 = 0.55

Two Shot: 6/13 = 0.46, Recall = 6/9 = 0.67

Three Shot: 6/10 = 0.6, Recall = 6/9 = 0.67


In [ ]:
print_trial('NCT02317744')
base = "Reported in Paper: Age, Sex, Race, Education, Binge Eating Frequency, Global eating-disorder psychopathology, \
Depression (BDI) scores, Body mass index, Percentage weight change from baseline"
print("")
print(textwrap.fill(base, width=150))

Trial ID: NCT02317744
 Title - Treatment of Binge Eating Disorder in Obesity: Naltrexone/ Bupropion Combination Versus Placebo Brief Summary - This study will test the
effectiveness of the combination of Naltrexone and Bupropion relative to placebo for reducing binge eating in persons with obesity and binge eating
disorder. Condition - Binge Eating Disorder,  Inclusion Criteria - * Binge eating disorder (full criteria as described in the American Psychiatric
Association Diagnostic and Statistical Manual of Mental Disorders, 5th edition) * BMI between 30 kg/m2 and 50 kg/m2 * Not taking anti-depressant
medications * Read English proficiently enough to read study assessments * Available for duration of treatment plus follow-up period * Able to travel
to study location (New Haven, CT) for monthly visits * Agree to study procedures Exclusion Criteria - * Medical status judged by study physician as
contraindication * History of seizures * Past or current anorexia nervosa, bulimia nervosa * C

**Example 3**

---
Zero Shot Matches: Age, PSA level, number of biopsy cores obtained, prior daily dietary supplement intake containing selenium, performance status (5/9)

One Shot Matches: Age, PSA level, number of biopsy cores, selenium supplementation history (4/9)

Two Shot Matches: Age, PSA level, Selenium Dietary Intake (3/9)

Three Shot Matches: Age, Race, PSA level, dietery supplement use (4/9)

Total features in reference = 9

Total features in responses = variable for each shot

Precision = TP / (TP + FP)

Recall = TP / (TP + FN)

Zero Shot: Precision = 5/10 = 0.5, Recall = 5/9 = 0.55

One Shot: Precision = 4/8 = 0.5 , Recall = 4/9 = 0.44

Two Shot: Precision = 3/7 = 0.42 , Recall = 3/9 = 0.33

Three Shot: Precision = 4/9 = 0.44 , Recall = 4/9 = 0.44

In [ ]:
print_trial('NCT00030901')
base = "Reported in Paper: Age, Race, Performance Status, Vitamin E Supplementation, Baseline PSA, Baseline Biopsy Requirement, BMI, \
Average Cores biopsied, Average of HGPIN-positive cores "
print("")
print(textwrap.fill(base, width=150))

Trial ID: NCT00030901
 Title - S9917, Selenium in Preventing Cancer in Patients With Neoplasia of the Prostate Brief Summary - RATIONALE: Chemoprevention therapy is the use
of certain drugs to try to prevent the development of cancer. The use of selenium may be an effective way to prevent prostate cancer in patients who
have neoplasia of the prostate.  PURPOSE: Randomized phase III trial to study the effectiveness of selenium in preventing prostate cancer in patients
who have neoplasia of the prostate. Condition - Precancerous/Nonmalignant Condition, Prostate Cancer,  Inclusion Criteria - STICS:  * Diagnosis of
high-grade prostatic intraepithelial neoplasia with no evidence of cancer    * Documented by a digital rectal exam and biopsy of the prostate with
transrectal ultrasound guidance (required if fewer than 6 cores obtained in biopsy) meeting one of the following conditions:      * Biopsy yielded
fewer than 10 cores within the past 24 months OR yielded more than 10 cores 6-24 months

### 2. GPT-4 Score Examples

In [15]:
# prompt: write a function that would take a string as input and return list of items separated by comma in the string. do this without using regular expressions. Make sure that no items from list "abc" are returned  in the final list. Remove any leading and trailing whitespace from the item. Make sure the comparison is case insensative.

def string_to_list(string):
  """
  Returns a list of items separated by comma in the string.

  Args:
      string: The string to extract items from.
      abc: A list of items to exclude from the final list.

  Returns:
      A list of items separated by comma in the string.
  """

  banned_list = ['continuous', 'categorical', 'custom', 'customized', '']

  items = string.split(',')
  final_items = []
  for item in items:
    item = item.strip()
    if item.lower() not in banned_list:
      final_items.append(item)

  return final_items



In [10]:
def get_gpt_4_score_examples(TrialId):

  testdf = df.loc[df['TrialID'] == TrialId]

  zero_shot = string_to_list(testdf[testdf['K-Shot'] == 'ZeroShot'].GPT_4_Response.values[0])
  one_shot = string_to_list(testdf[testdf['K-Shot'] == 'OneShot'].GPT_4_Response.values[0])
  two_shot = string_to_list(testdf[testdf['K-Shot'] == 'TwoShot'].GPT_4_Response.values[0])
  three_shot = string_to_list(testdf[testdf['K-Shot'] == 'ThreeShot'].GPT_4_Response.values[0])

  return zero_shot, one_shot, two_shot, three_shot

In [16]:
# prompt: write code to ask OpenAI GPT-4 model to score similarity between reference and each response

from openai import OpenAI
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY_BS1")
def get_openai_score(base, ze, on, tw, th):

  client = OpenAI()

  prompt = (
      f"We are evaluating four AI models' responses based on their similarity to a reference string.\n"
      f"Reference string: {base}\n"
      f"Zero Shot Response: {ze}\n"
      f"One Shot Response: {on}\n"
      f"Two Shot Response: {tw}\n"
      f"Three Shot Response: {th}\n"
      f"Your task is to find how many features in each response matches to any of the item in the reference string."
      f"Make sure you take underlying semantic and contexual similarity of each feature into account."
      f"Take each feature in a response list, then see if there is something similar in the reference list. Do this for all items in the response list, and for all responses. Perform case insensitive matching."
      f"Only consider features that match exactly, and that matches partially based on contextuality/semantics. Consider partial matches as a match."
      f"Calculate the number of total matches for each response, don't make mistakes in your count - \n"
      f"Return them in a table format with three columns: Response Name, Number of Matches / Number of total features in reference (show in a/b format, don't make it decimal), and List of Matched features."
      f"Make sure to revise, re-check and count the number of matches for each response."
      f"Don't give any additional explanations"
  )

  completion = client.chat.completions.create(
    model="gpt-4-0125-preview",
    temperature=0,
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant with 10 years of experience in clinical domain and clinical trial design.\
    Answer the question based on the context below and your knowledge on clinical trial design, and clinical domain."},
      {"role": "user", "content": prompt}
    ]
  )

  print(completion.choices[0].message.content)


**Example 1**


---



In [17]:
base = "Age, Race, Sex, ECOG Performance Status, Prior Therapy (Surgery/Radiotherapy/Bisphosphonates), \
Bone Metastatis, RECIST-evaluable diseases, Disease Sites (Lymph/Pelvis/Adrenal Gland/Visceral-Lung/Visceral-Liver/Visceral-Diaphragm)"
base_list = string_to_list(base)
ze, on, tw, th = get_gpt_4_score_examples('NCT00385580')

In [18]:
get_openai_score(base, ze, on, tw, th)

| Response Name   | Number of Matches / Number of total features in reference | List of Matched features                                                                                   |
|-----------------|------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| Zero Shot       | 3/8                                                        | Age, Presence of bone metastases, Performance status                                                       |
| One Shot        | 5/8                                                        | Age, Sex: Male, Eastern Cooperative Oncology (ECOG) performance status, Presence of Metastatic Disease     |
| Two Shot        | 6/8                                                        | Age, Sex: Male, Baseline Prostate Specific Antigen (PSA) Level, Metastatic Sites, Eastern Cooperative Oncology Group (ECOG) Performance Status                 

**Example 2**


---



In [ ]:
base = "Age, Sex, Race, Education, Binge Eating Frequency, Global eating-disorder psychopathology, \
Depression (BDI) scores, Body mass index, Percentage weight change from baseline"
base_list = string_to_list(base)
ze, on, tw, th = get_gpt_4_score_examples('NCT02317744')

In [ ]:
get_openai_score(base, ze, on, tw, th)

| Response Name   | Number of Matches / Number of total features in reference | List of Matched features                                                                                   |
|-----------------|------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| Zero Shot       | 5/9                                                        | Age, sex, BMI, Depression (baseline depression score), Binge Eating Frequency (number of binge eating episodes per week) |
| One Shot        | 4/9                                                        | Age, Sex, BMI (BMI (kg/m2)), Depression (Depression scale score)                                            |
| Two Shot        | 6/9                                                        | Age, Sex, Race, BMI (BMI (kg/m2)), Depression (Current Depression Scale Score), Binge Eating Frequency (Number of Binge Eating Episodes per Week

**Example 3**


---



In [ ]:
base = "Reported in Paper: Age, Race, Performance Status, Vitamin E Supplementation, Baseline PSA, Baseline Biopsy Requirement, BMI, \
Average Cores biopsied, Average of HGPIN-positive cores"
base_list = string_to_list(base)
ze, on, tw, th = get_gpt_4_score_examples('NCT00030901')

In [ ]:
get_openai_score(base, ze, on, tw, th)

| Response Name  | Number of Matches / Number of total features in reference | List of Matched features                                                                                   |
|----------------|------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| Zero Shot      | 5/9                                                        | Age, PSA level, performance status (SWOG), number of biopsy cores obtained, presence of high-grade prostatic intraepithelial neoplasia |
| One Shot       | 3/9                                                        | Age, PSA Level, Number of Biopsy Cores                                                                      |
| Two Shot       | 4/9                                                        | Age, PSA Level, Number of Prior Prostate Biopsies, High-Grade Prostatic Intraepithelial Neoplasia Presence                             

# Comparison between Human and GPT-4 Eval

Example | Response  | Human Precision | Human Recall  | GPT Precision | GPT Recall
---     | ---       | ---             | ---           | ---           | ---
Example 1 | Zero Shot | 0.31| 0.63| 0.31 | 0.63
          | One Shot | 0.50| 0.63| 0.60 | 0.75
          | Two Shot | 0.56| 0.63| 0.67 | 0.75
          | Three Shot | 0.44| 0.50| 0.55 | 0.64
Example 2 | Zero Shot | 0.26| 0.55| 0.26 | 0.55
          | One Shot | 0.38| 0.55| 0.31 | 0.44
          | Two Shot | 0.46| 0.67| 0.46 | 0.67
          | Three Shot | 0.60| 0.67| 0.70 | 0.78
Example 3 | Zero Shot | 0.50| 0.55| 0.50 | 0.55
          | One Shot | 0.50 | 0.44| 0.38 | 0.33
          | Two Shot | 0.42 | 0.33| 0.57 | 0.44
          | Three Shot | 0.44 | 0.44| 0.44 | 0.44

# Notes



*   Generate Few-Shot random examples for all 100 test-trials using GPT-4 [Dataframe Format: ID/Base/Zero-Shot/1-Shot/2-Shot/3-Shot] (Done)
*   Compare performance with zero-shot examples (Done)
*   Hand score a few (3) trial generations - both for zero/few-shot examples (Done)
*   Make GPT-4 Score matching keywords (Done)
*   Paper structure, what to submit, draft outline discuss


- https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples/
- https://python.langchain.com/docs/integrations/chat/openai/
- https://platform.openai.com/docs/guides/prompt-engineering/six-strategies-for-getting-better-results
- https://neurips.cc/Conferences/2023/CallForDatasetsBenchmarks

